In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')

Connecting celiib@10.28.0.34:3306


In [3]:
#ta3p100.ProofreadLabelOrphan() & "segment_id=648518346341352006"

In [4]:
#ta3p100.CleansedMeshOrphan()

In [5]:
@schema
class CoarseLabelOrphan(dj.Computed):
    definition = """
    # Vertex labels for pinky.ProofreadLabelOrphan did not correctly match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.ProofreadLabelOrphan
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.ProofreadLabelOrphan
    """
    
    key_source = pinky.ProofreadLabelOrphan & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.ProofreadLabelOrphan & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.Decimation35OrphanStitched & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [6]:
start = time.time()
CoarseLabelOrphan.populate(reserve_jobs=True)
print(time.time() - start)

648518346341394638
Segment 648518346341394638 vertex labels regenerated in: 0.8631160259246826 seconds.
648518346341395530
Segment 648518346341395530 vertex labels regenerated in: 0.8710589408874512 seconds.
648518346341397329
Segment 648518346341397329 vertex labels regenerated in: 1.1081628799438477 seconds.
648518346341398705
Segment 648518346341398705 vertex labels regenerated in: 0.7476010322570801 seconds.
648518346341399382
Segment 648518346341399382 vertex labels regenerated in: 0.7855334281921387 seconds.
648518346341400285
Segment 648518346341400285 vertex labels regenerated in: 0.6964263916015625 seconds.
648518346341401368
Segment 648518346341401368 vertex labels regenerated in: 0.6804556846618652 seconds.
648518346341402446
Segment 648518346341402446 vertex labels regenerated in: 0.7528557777404785 seconds.
648518346341403291
Segment 648518346341403291 vertex labels regenerated in: 0.7069962024688721 seconds.
648518346341404064
Segment 648518346341404064 vertex labels rege

In [7]:
(schema.jobs & "table_name='__coarse_label_orphan'").delete()

In [8]:
ta3p100.CoarseLabelOrphan()

NameError: name 'ta3p100' is not defined

In [ ]:
ta3p100.SegmentExclude()